In [ ]:
import numpy as np
import pandas as pd
import scipy
import itertools
import time
import sys
import datetime
import os
import validation_functions as vf

from kesi._verbose import (VerboseFFR,
                           LinearMixture,
                           LoadableVerboseFFR)
from kesi._engine import _LinearKernelSolver
sys.path.append('..')
from FEM.fem_sphere_gaussian import (SomeSphereGaussianSourceFactory3D,
                                     SomeSphereGaussianSourceFactoryOnlyCSD)
from _common_new import altitude_azimuth_mesh
import plotting_functions as pf
try:
    from joblib import Parallel, delayed
    import multiprocessing
    NUM_CORES = multiprocessing.cpu_count() - 1
    PARALLEL_AVAILABLE = True
except ImportError:
    PARALLEL_AVAILABLE = False

MeasurementManagerBase = VerboseFFR.MeasurementManagerBase

In [ ]:
def makemydir(directory):
    """
    Creates directory if it doesn't exist
    """
    try:
        os.makedirs(directory)
    except OSError:
        pass
    os.chdir(directory)

In [ ]:
def generate_EEG_electrodesRPI(nr_theta, nr_phi, r):
    '''Determine positions of electrodes - EEG
    '''
    theta, phi, r = np.meshgrid(np.linspace(-0.5*np.pi, 0.5*np.pi, nr_theta),
                                np.linspace(0, 2*np.pi, nr_phi),
                                r)
    ELE_X = -r*np.cos(theta)*np.cos(phi)
    ELE_Y = r*np.cos(theta)*np.sin(phi)
    ELE_Z = r*np.sin(theta)
    ELECTRODES = pd.DataFrame({'X': ELE_X.flatten(),
                               'Y': ELE_Y.flatten(),
                               'Z': ELE_Z.flatten()})
    ELECTRODES_NR = ELE_X.size
    return ELECTRODES, ELECTRODES_NR

def generate_ECoG_electrodes(nr_x, nr_y, Z):
    '''Determine positions of electrodes - ECoG
    '''
    xx, yy, zz = np.meshgrid(np.linspace(-0.035, 0.035, nr_x),
                             np.linspace(-0.035, 0.035, nr_y),
                             [-Z])
    ELE_PLANE = np.zeros([xx.size, 3])
    ELE_PLANE[:, 0] = xx.flatten()
    ELE_PLANE[:, 1] = yy.flatten()
    ELE_PLANE[:, 2] = zz.flatten()
    ELE_SPHERE = np.array([Z*ELE_PLANE[i]/np.linalg.norm(ELE_PLANE[i]) for i in range(xx.size)])
    ELECTRODES = pd.DataFrame({'X': ELE_SPHERE[:, 0],
                               'Y': ELE_SPHERE[:, 1],
                               'Z': ELE_SPHERE[:, 2]})
    ELECTRODES_NR = xx.size
    return ELECTRODES, ELECTRODES_NR

In [ ]:
start_time = time.time()
meshname = 'four_spheres_gaussian_1000_deg_1'
MESHFILE = '/home/mbejtka/Data_Kuba/' + meshname + '.npz'
now = datetime.datetime.today() 
nTime = now.strftime('%Y-%m-%d_%H-%M-%S')
save_path = sys.path[0] + '/results/' + nTime
makemydir(save_path)
forward_model = 'four_spheres'

factory = SomeSphereGaussianSourceFactory3D(MESHFILE)
print("Loading data --- %s seconds ---" % (time.time() - start_time))

dst = factory.R[1] - factory.R[0]
sources = [factory(r, altitude, azimuth)
           for altitude, azimuth in altitude_azimuth_mesh(-np.pi/2,
                                                          dst/factory.scalp_radius)
           for r in factory.R]
setup_dict = {'meshname': meshname, 'path': save_path, 'sources': sources}
print('Number of sources: ', len(sources))
print("Sources --- %s seconds ---" % (time.time() - start_time))

In [ ]:
ele_type = 'ECoG'

if ele_type == 'EEG':
    ELECTRODES, ELECTRODES_NR = generate_EEG_electrodesRPI(15,
                                                           15,
                                                           [factory.scalp_radius])
elif ele_type == 'ECoG':
    ELECTRODES, ELECTRODES_NR = generate_ECoG_electrodes(8, 8, factory.R.max())
setup_dict.update({'ele_type': ele_type, 'ELECTRODES': ELECTRODES})

In [ ]:
# Estimating points    
r = factory.scalp_radius
X, Y, Z = np.meshgrid(np.linspace(-r, r, 30),
                      np.linspace(-r, r, 30),
                      np.linspace(-r, r, 30))
IDX = X**2 + Y**2 + Z**2 <=r**2
EST_X = X[IDX]
EST_Y = Y[IDX]
EST_Z = Z[IDX]
EST_POINTS =pd.DataFrame({'X': EST_X.flatten(),
                          'Y': EST_Y.flatten(),
                          'Z': EST_Z.flatten()})
setup_dict.update({'X':X, 'Y':Y, 'Z':Z, 'IDX': IDX, 'EST_POINTS': EST_POINTS})

In [ ]:
measurement_manager = vf.MeasurementManager(ELECTRODES, space='potential')
measurement_manager_basis = vf.MeasurementManager(EST_POINTS, space='csd')

In [ ]:
# Create reconstructor
reconstructor_filename = save_path +'/Reconstructor.npz'
reconstructor = VerboseFFR(sources, measurement_manager)
reconstructor.save(reconstructor_filename)
print("Reconstructor --- %s seconds ---" % (time.time() - start_time))

In [ ]:
factoryCSD = SomeSphereGaussianSourceFactoryOnlyCSD(MESHFILE)
dst = factoryCSD.R[1] - factoryCSD.R[0]
sourcesCSD = [factoryCSD(r, altitude, azimuth)
              for altitude, azimuth in altitude_azimuth_mesh(-np.pi/2,
                                                          dst/factory.scalp_radius)
              for r in factoryCSD.R]

In [ ]:
loadable_reconstructor = LoadableVerboseFFR(reconstructor_filename, sourcesCSD, measurement_manager)
kernel = loadable_reconstructor.kernel
cross_kernel = loadable_reconstructor.get_kernel_matrix(measurement_manager_basis)
cross_reconstructor = loadable_reconstructor._CrossKernelReconstructor(_LinearKernelSolver(kernel), cross_kernel)

In [ ]:
cross_reconstructor_filename = save_path + '/CrossReconstructor.npz'
cross_reconstructor.save(cross_reconstructor_filename)

In [ ]:
potential = [measurement_manager.probe(source) for source in sources]

In [ ]:
true_csd = [measurement_manager_basis.probe(source) for source in sourcesCSD]

In [ ]:
setup_dict.update({'POTENTIAL': potential, 'TRUE_CSD': true_csd})
np.savez_compressed(save_path + '/Setup_info.npz', **setup_dict)

In [ ]:
eigensources, eigenvalues, eigenvectors = vf.calculate_eigensources(kernel, cross_kernel, regularization_parameter=0)
projection = vf.csd_into_eigensource_projection(true_csd, eigensources)
np.savez_compressed(save_path + '/Spectral_decomposition.npz',
                    EIGENSOURCES=eigensources,
                    EIGENVALUES=eigenvalues, EIGENVECTORS=eigenvectors,
                    PROJECTION=projection)

In [ ]:
eigen_parameters = eigenvalues[np.where(eigenvalues > 0)]

In [ ]:
#l_lambd, lcurve_axis, curve_surf, lambdas = [L_curve(kernel, potential[src_nr], lambdas=np.logspace(-20, -1, 30))
#                                             for src_nr in range(len(sources))]

In [ ]:
import plotting_functions as pf
%load_ext autoreload
%autoreload 2

In [ ]:
from imp import reload 
reload(pf)
reload(vf)

In [ ]:
src_nr = 0
potential_noise, noise = vf.add_noise(potential[src_nr], seed=0, level=30)
l_lambd, lcurve_axis, curve_surf, lambdas = vf.L_curve(kernel, potential_noise, lambdas=eigen_parameters)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(potential[src_nr])
plt.plot(potential_noise)

In [ ]:
plt.figure()
plt.plot(lcurve_axis[1], lcurve_axis[0], '.')
plt.plot(lcurve_axis[1, np.where(lambdas==l_lambd)], lcurve_axis[0, np.where(lambdas==l_lambd)], 'o', label='lambda')
plt.title('L-curve')
plt.legend()
plt.savefig(save_path + '/L-curve.png')

In [ ]:
reg_params = np.logspace(-20, 1, 30)
EST_CSD_0, indx_rp0, EE_0, rp0 = vf.estimate_csd(cross_reconstructor, potential_noise.reshape(1,len(potential[src_nr])),
                                        regularization_parameters=reg_params)

In [ ]:
reg_params[indx_rp0]

In [ ]:
est_csd_noise = cross_reconstructor(potential_noise, reg_params[indx_rp0])

In [ ]:
est_csd_noise_0 = cross_reconstructor(potential_noise, np.array([0]))

In [ ]:
plt.figure(figsize=(6, 6))
ax = plt.subplot(111)
t_max = np.max(abs(est_csd_noise))
pf.make_subplot(ax, pf.values_in_a_grid(X, est_csd_noise, IDX), 'csd', X, Y, Z, t_max, idx=15, fig_title='CV')

In [ ]:
plt.figure(figsize=(6, 6))
ax = plt.subplot(111)
t_max = np.max(abs(est_csd_noise_0))
pf.make_subplot(ax, pf.values_in_a_grid(X, est_csd_noise_0, IDX), 'csd', X, Y, Z, t_max, idx=15, fig_title='Without regularization')

In [ ]:
est_csd_noise_lc = cross_reconstructor(potential_noise, l_lambd)

In [ ]:
plt.figure(figsize=(6, 6))
ax = plt.subplot(111)
t_max = np.max(abs(est_csd_noise_lc))
pf.make_subplot(ax, pf.values_in_a_grid(X, est_csd_noise_lc, IDX), 'csd', X, Y, Z, t_max, idx=15, fig_title='L-curve')

In [ ]:
EST_CSD_LC, indx_rpLC, EE_LC, rpLC = vf.estimate_csd(cross_reconstructor, potential,
                                        regularization_parameters=np.array([l_lambd]))

In [ ]:
EST_CSD_0, indx_rp0, EE_0, rp0 = vf.estimate_csd(cross_reconstructor, potential,
                                        regularization_parameters=np.array([0]))

In [ ]:
regularization_parameters = eigen_parameters
EST_CSD, indx_rp, EE, rp = vf.estimate_csd(cross_reconstructor, potential,
                                        regularization_parameters=regularization_parameters)

In [ ]:
indx_rp.shape

In [ ]:
src_nr = 0
EST_CSD2, indx_rp2, EE2, rp2 = vf.estimate_csd(cross_reconstructor, potential_noise.reshape(1,len(potential[src_nr])),
                                        regularization_parameters=eigen_parameters)

In [ ]:
np.logspace(-5, 5, 100)[indx_rp2]

In [ ]:
plt.figure()
plt.plot(np.logspace(-5, 5, 100), EE2, '.', label='regular')
plt.plot(eigen_parameters, EEn, '.', label='eigenV')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Regularization parameter')
plt.ylabel('Error in potential space')
plt.legend()
plt.savefig(save_path + '/Pots_Error_eigenvalues_vs_regular_params.png')

In [ ]:
regularization_parameters = eigen_parameters
EST_CSDn, indx_rpn, EEn, rpn = vf.estimate_csd(cross_reconstructor, noise.reshape(1,len(potential[src_nr])),
                                        regularization_parameters=regularization_parameters)

In [ ]:
rpn

In [ ]:
np.savez_compressed(save_path + '/Estimation.npz', EST_CSD_CV=EST_CSD, indx_rp=indx_rp, EE=EE,
                    LAMBD=rp, REG_PARAMS=regularization_parameters, L_LAMBD=l_lambd, LCURVE=lcurve_axis,
                    EST_CSD_0=EST_CSD_0, EE_0=EE_0, EST_CSD_LC=EST_CSD_LC, EE_LC=EE_LC)

In [ ]:
src_nr = 0
rms = np.zeros(len(eigen_parameters))
rdm = np.zeros(len(eigen_parameters))
mag = np.zeros(len(eigen_parameters))
for j, value in enumerate(eigen_parameters):
    est_csd_ev = cross_reconstructor(potential[src_nr], np.array(value))
    rms[j] = vf.calculate_rms(true_csd[src_nr], est_csd_ev)
    rdm[j] = vf.calculate_rdm(true_csd[src_nr], est_csd_ev)
    mag[j] = vf.calculate_mag(true_csd[src_nr], est_csd_ev)
np.savez_compressed(save_path + '/GlobalErrors.npz', RMS=rms, RDM=rdm, MAG=mag)

In [ ]:
src_nr = 0
#eigen_parameters2 = np.logspace(-5, 5, 64)
eigen_parameters2 = eigen_parameters
rms_n = np.zeros(len(eigen_parameters2))
rdm_n = np.zeros(len(eigen_parameters2))
mag_n = np.zeros(len(eigen_parameters2))
for j, value in enumerate(eigen_parameters2):
    est_csd_ev = cross_reconstructor(potential_noise, np.array(value))
    rms_n[j] = vf.calculate_rms(true_csd[src_nr], est_csd_ev)
    rdm_n[j] = vf.calculate_rdm(true_csd[src_nr], est_csd_ev)
    mag_n[j] = vf.calculate_mag(true_csd[src_nr], est_csd_ev)
#np.savez_compressed(save_path + '/GlobalErrorsNoise.npz', RMS=rms_n, RDM=rdm_n, MAG=mag_n)

In [ ]:
src_nr = 0
#eigen_parameters2 = np.logspace(-5, 5, 64)
eigen_parameters2 = eigen_parameters
rms_noise = np.zeros(len(eigen_parameters2))
rdm_noise = np.zeros(len(eigen_parameters2))
mag_noise = np.zeros(len(eigen_parameters2))
for j, value in enumerate(eigen_parameters2):
    est_csd_ev = cross_reconstructor(noise, np.array(value))
    rms_noise[j] = vf.calculate_rms(true_csd[src_nr], est_csd_ev)
    rdm_noise[j] = vf.calculate_rdm(true_csd[src_nr], est_csd_ev)
    mag_noise[j] = vf.calculate_mag(true_csd[src_nr], est_csd_ev)

In [ ]:
pf.plot_potential_ele(ELECTRODES.X, ELECTRODES.Y, ELECTRODES.Z, potential[src_nr], 'Potential without noise', save_path)

In [ ]:
pf.plot_potential_ele(ELECTRODES.X, ELECTRODES.Y, ELECTRODES.Z, potential_noise, 'Potential with noise', save_path)

In [ ]:
pf.plot_potential_ele(ELECTRODES.X, ELECTRODES.Y, ELECTRODES.Z, noise, 'Noise', save_path)

In [ ]:
layer = 15
src_nr = 0
pf.generate_Picard_subplot(eigenvalues, eigenvectors, true_csd[src_nr], potential[src_nr], EE[:, src_nr],
                           rms, rdm, mag, eigen_parameters[indx_rp[src_nr]], forward_model, src_nr, save_path,
                           ELECTRODES.X, ELECTRODES.Y, ELECTRODES.Z,
                           sources[src_nr].x, sources[src_nr].y, sources[src_nr].z, X, Y, Z, IDX, layer)

In [ ]:
src_nr = 0
pf.generate_Picard_subplot(eigenvalues, eigenvectors, true_csd[src_nr], potential_noise, EE2,
                           rms_n, rdm_n, mag_n, eigen_parameters[indx_rp2[src_nr]], forward_model, src_nr, save_path,
                           ELECTRODES.X, ELECTRODES.Y, ELECTRODES.Z,
                           sources[src_nr].x, sources[src_nr].y, sources[src_nr].z, X, Y, Z, IDX, layer)

In [ ]:
src_nr = 0
pf.generate_Picard_subplot(eigenvalues, eigenvectors, true_csd[src_nr], noise, EEn,
                           rms_noise, rdm_noise, mag_noise, eigen_parameters[indx_rpn[src_nr]], forward_model, src_nr, save_path,
                           ELECTRODES.X, ELECTRODES.Y, ELECTRODES.Z,
                           sources[src_nr].x, sources[src_nr].y, sources[src_nr].z, X, Y, Z, IDX, layer)